<a href="https://colab.research.google.com/github/ethanduncan65/MLB-Series-Analyzer/blob/main/MLB_Series_Analyzer_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLB Series Analyzer 


#### This is a query tool that analyzes an MLB team's season at the series level. Run the first cell below to install the MLB-Stats API package. Next, select your desired season and team, and then run that cell.

#### Data Source: https://github.com/toddrob99/MLB-StatsAPI 

In [ ]:
pip install MLB-StatsAPI

In [ ]:
# import dependencies 
from datetime import datetime, timedelta
import pandas as pd
import statsapi

#@title Select Year and Team 
Year = 2021 #@param ["2021", "2020", "2019", "2018", "2017", "2016", "2015", "2014", "2013", "2012", "2011", "2010", "2009", "2008", "2007", "2006", "2005"] {type:"raw"}
Team = "Seattle Mariners" #@param ["Arizona Diamondbacks", "Atlanta Braves", "Baltimore Orioles", "Boston Red Sox", "Chicago Cubs", "Chicago White Sox", "Cincinnati Reds", "Cleveland Indians", "Colorado Rockies", "Detroit Tigers", "Houston Astros", "Kansas City Royals", "Los Angeles Angels", "Los Angeles Dodgers", "Miami Marlins", "Milwaukee Brewers", "Minnesota Twins", "New York Mets", "New York Yankees", "Oakland Athletics", "Philadelphia Phillies", "Pittsburgh Pirates", "San Diego Padres", "Seattle Mariners", "San Francisco Giants", "St. Louis Cardinals", "Tampa Bay Rays", "Texas Rangers", "Toronto Blue Jays", "Washinton Nationals"]

# determine start & end date for selected year
startDate = str('01/01/' + str(Year))
endDate =  str('12/31/' + str(Year))

# query game data based on selected year & team
team_id = statsapi.lookup_team(Team)
team_sched = statsapi.schedule(start_date=startDate, 
                               end_date=endDate, 
                               team=team_id[0]['id'])



# create game log dataframe
df_gameLog = pd.DataFrame(columns = ['Series #', 'Date', 'Opponent', 'R', 'RA', 
                                     'Rdiff', 'Outcome', 'Home Team', 
                                     'Away Team', 'Home Score', 'Away Score', 
                                     'Location', 'Venue'], 
                          index = range(0, 200)) 
# loop through each game and fill out game log dataframe
for i in range(0, len(team_sched)):
  # skip over spring non-regular season games
  i_gameType = team_sched[i].get('game_type')
  if i_gameType != 'R':
    continue
  # skip over any incomplete games
  i_gameStatus = team_sched[i].get('status')
  if i_gameStatus != 'Final':
    continue
  # parse API call and tfx to df
  i_homeName = team_sched[i].get('home_name')
  i_awayName = team_sched[i].get('away_name')
  df_gameLog.at[i, 'Date'] = team_sched[i].get('game_date')
  df_gameLog.at[i, 'Venue'] = team_sched[i].get('venue_name')
  df_gameLog.at[i, 'Home Team'] = team_sched[i].get('home_name')
  df_gameLog.at[i, 'Away Team'] = team_sched[i].get('away_name')
  df_gameLog.at[i, 'Home Score'] = team_sched[i].get('home_score')
  df_gameLog.at[i, 'Away Score'] = team_sched[i].get('away_score')
  # determine data based on home/away team
  if i_homeName==Team:
    df_gameLog.at[i, 'Opponent'] = i_awayName
    df_gameLog.at[i, 'Location'] = 'Home'
    df_gameLog.at[i, 'R'] = team_sched[i].get('home_score')
    df_gameLog.at[i, 'RA'] = team_sched[i].get('away_score')
    df_gameLog.at[i, 'Rdiff'] = team_sched[i].get('home_score')-team_sched[i].get('away_score')
  else:
    df_gameLog.at[i, 'Opponent'] = i_homeName
    df_gameLog.at[i, 'Location'] = 'Away'
    df_gameLog.at[i, 'R'] = team_sched[i].get('away_score')
    df_gameLog.at[i, 'RA'] = team_sched[i].get('home_score')
    df_gameLog.at[i, 'Rdiff'] = team_sched[i].get('away_score')-team_sched[i].get('home_score')
  #
  if df_gameLog.iloc[i]['Rdiff'] > 0:
    df_gameLog.at[i, 'Outcome'] = 'W'
  else:
    df_gameLog.at[i, 'Outcome'] = 'L'
  # _______


# drop any empty rows
df_gL = df_gameLog.dropna(thresh=2)
df_gameLog_clean = df_gL.reset_index()

series_num = 0 
# loop through each game and assign it a series number 
for k in range(0, len(df_gameLog_clean)):
  # ...
  if k==0:
    series_num+=1
    # assign series number 
    df_gameLog_clean.at[k, 'Series #'] = series_num
  # else, determine if k'th game is start of new series
  else:
    newSeries = df_gameLog_clean.iloc[k-1]['Opponent']!=df_gameLog_clean.iloc[k]['Opponent']
    # trigger new series if ballpark is changed
    if df_gameLog_clean.iloc[k-1]['Venue']!=df_gameLog_clean.iloc[k]['Venue']:
      newSeries = True
    # increment counter for new series
    if newSeries:
      series_num+=1
    # assign series number 
    df_gameLog_clean.at[k, 'Series #'] = series_num

# create final dataframe
df_final = pd.DataFrame(columns = ['Series #', 'Start Date', 'End Date', 
                                   'Opponent', 'Result', 
                                   'Location', 'GP', 'W', 
                                   'L', 'Rdiff'], 
                        index = range(0, series_num))
# loop through each series and fill out final dataframe filtering by series_num
for s in range(0, series_num):
  # create filtered df for each series
  df_s = df_gameLog_clean[df_gameLog_clean['Series #'] == s+1]
  #
  df_final.at[s, 'Series #'] = s+1
  df_final.at[s, 'Start Date'] = df_s.iloc[0]['Date']
  df_final.at[s, 'End Date'] = df_s.iloc[len(df_s)-1]['Date']
  df_final.at[s, 'Opponent'] = df_s.iloc[0]['Opponent']
  df_final.at[s, 'Location'] = df_s.iloc[0]['Location']
  df_final.at[s, 'GP'] = len(df_s)
  df_final.at[s, 'W'] = len(df_s[df_s['Outcome'] == 'W'])
  df_final.at[s, 'L'] = len(df_s[df_s['Outcome'] == 'L'])
  df_final.at[s, 'Rdiff'] = df_s['Rdiff'].sum()

# determine result for each series
for r in range(0, len(df_final)):
  # swept
  if df_final.iloc[r]['W']==df_final.iloc[r]['GP']:
    df_final.at[r, 'Result'] = 'Swept'
  # got swept
  elif df_final.iloc[r]['L']==df_final.iloc[r]['GP']:
    df_final.at[r, 'Result'] = 'Got Swept'
  # won
  elif df_final.iloc[r]['W']>df_final.iloc[r]['L']:
    df_final.at[r, 'Result'] = 'Won'
  # lost
  elif df_final.iloc[r]['W']<df_final.iloc[r]['L']:
    df_final.at[r, 'Result'] = 'Lost'
  else:
    df_final.at[r, 'Result'] = 'Split'

# color pallete: https://matplotlib.org/stable/gallery/color/named_colors.html 
def color(val):
  if val == 'Swept':
    color = 'green'
  elif val == 'Won':
    color = 'mediumspringgreen'
  elif val == 'Lost':
    color = 'salmon'
  elif val == 'Got Swept':
    color = 'red'
  else:
    color = 'lightgray'
  return 'background-color: %s' % color

# apply condintional formatting final dataframe
df9 = df_final.set_index('Series #')
df9.style.applymap(color, subset=['Result'])
